In [8]:
! pip install torch


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
! pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### This Script Has Code to Generate Embeddings using 3 Models namely GLOVE,WORD2VEC,FASTTEXT

In [13]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import numpy as np

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
current_folder = r'C:\Users\student\Desktop\Datasets\sample_data'
images_folder = os.path.join(current_folder, "Images")
labels_folder = os.path.join(current_folder, "tamil_labels_300_proofread")
output_folder = os.path.join(current_folder, "embeddings_300_proofread")

In [6]:
# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

##### Loading Muril Model for embedding Generation

In [4]:
# Load MURIL model and tokenizer
model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

c:\Users\mhanj\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

c:\Users\mhanj\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'



pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

c:\Users\mhanj\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


NameError: name 'model' is not defined

In [6]:
# Function to generate embeddings
def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use mean pooling to get sentence embedding
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embedding

In [7]:
for image_file in tqdm(os.listdir(images_folder)):
    image_name = os.path.splitext(image_file)[0]
    label_file = os.path.join(labels_folder, f"{image_name}.txt")
    
    # Check if the label file exists
    if not os.path.exists(label_file):
        print(f"Label file not found for {image_file}")
        continue
    
    # Read the label (description)
    with open(label_file, "r", encoding="utf-8") as f:
        description = f.read().strip()
    
    # Generate embedding
    embedding = generate_embedding(description)
    
    # Save embedding
    output_file = os.path.join(output_folder, f"{image_name}.npy")
    np.save(output_file, embedding)

100%|██████████| 300/300 [00:18<00:00, 16.28it/s]


In [8]:
import random

In [9]:
# Print sample embeddings
def print_sample_embeddings(folder, num_samples=3):
    print(f"\nPrinting {num_samples} sample embeddings:")
    embedding_files = [f for f in os.listdir(folder) if f.endswith('.npy')]
    
    if not embedding_files:
        print("No embedding files found.")
        return
    
    samples = random.sample(embedding_files, min(num_samples, len(embedding_files)))
    
    for sample in samples:
        file_path = os.path.join(folder, sample)
        embedding = np.load(file_path)
        print(f"\nFile: {sample}")
        print(f"Embedding shape: {embedding.shape}")
        print(f"First 10 values: {embedding[:10]}")
        print(f"Min value: {np.min(embedding)}, Max value: {np.max(embedding)}")

# Call the function to print sample embeddings
print_sample_embeddings(output_folder)


Printing 3 sample embeddings:

File: image_00234.npy
Embedding shape: (768,)
First 10 values: [-0.00110763  0.00386116 -0.00264634 -0.01812263 -0.00447125  0.00063988
 -0.00550396 -0.00915958 -0.0029088  -0.00025061]
Min value: -0.5953381061553955, Max value: 0.15569542348384857

File: image_00243.npy
Embedding shape: (768,)
First 10 values: [-0.00027811  0.00291532 -0.00330758 -0.01658205 -0.00499371 -0.00028355
 -0.00818089 -0.0056842  -0.00224374 -0.00181728]
Min value: -0.5964596271514893, Max value: 0.15918126702308655

File: image_00109.npy
Embedding shape: (768,)
First 10 values: [-0.00222336  0.00096072 -0.00247957 -0.01388167 -0.00400691  0.00228519
 -0.00787494 -0.00564079 -0.00286657 -0.00044924]
Min value: -0.5972206592559814, Max value: 0.15914572775363922


#### FastText Model

In [17]:
import shutil
import os
import requests
from tqdm import tqdm
from gensim.models.fasttext import FastText

In [5]:
def download_file(url, filename):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB

    with open(filename, 'wb') as file, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as progress_bar:
        for data in response.iter_content(block_size):
            size = file.write(data)
            progress_bar.update(size)


In [ ]:
os.makedirs('tamil_models', exist_ok=True)
# Download FastText model for Tamil
print("Downloading FastText model for Tamil...")
fasttext_url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.bin.gz"
fasttext_gz_path = 'tamil_models/cc.ta.300.bin.gz'
fasttext_bin_path = 'tamil_models/cc.ta.300.bin'

download_file(fasttext_url, fasttext_gz_path)

print("Extracting FastText model...")
with gzip.open(fasttext_gz_path, 'rb') as f_in:
    with open(fasttext_bin_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

os.remove(fasttext_gz_path)  # Remove the .gz file after extraction

print("Loading FastText model...")

In [18]:
fasttext_bin_path = 'tamil_models/cc.ta.300.bin'
fasttext_model = FastText.load_fasttext_format(fasttext_bin_path)
fasttext_model.save('tamil_models/fasttext_tamil.model')
print("FastText model saved.")

C:\Users\student\AppData\Local\Temp\ipykernel_8532\94384607.py:2: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fasttext_model = FastText.load_fasttext_format(fasttext_bin_path)


FastText model saved.


In [35]:
current_folder = r'C:\Users\student\Desktop\Datasets\sample_data'
images_folder = os.path.join(current_folder, "Images")
labels_folder = os.path.join(current_folder, "tamil_labels_300_proofread")
output_folder = os.path.join(current_folder, "embeddings_300_proofread_fasttext")

In [33]:
def generate_embedding_fasttext(text, model):
    # Tokenize the text into words
    words = text.split()
    
    # Generate embeddings for each word and take the mean
    embeddings = []
    for word in words:
        if word in model.wv:
            embeddings.append(model.wv[word])
        else:
            # If word is not found, use a zero vector
            embeddings.append(np.zeros(model.vector_size))
    
    # Calculate the mean of the word embeddings to get a sentence embedding
    embedding = np.mean(embeddings, axis=0)
    return embedding


In [25]:
#This version of gensim is needed to run the key_to_index command
! pip install gensim==3.8.3


     ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
     --------------------------------------- 0.0/23.4 MB 991.0 kB/s eta 0:00:24
      --------------------------------------- 0.3/23.4 MB 4.1 MB/s eta 0:00:06
     - -------------------------------------- 0.6/23.4 MB 5.8 MB/s eta 0:00:04
     - -------------------------------------- 1.0/23.4 MB 6.6 MB/s eta 0:00:04
     -- ------------------------------------- 1.5/23.4 MB 7.4 MB/s eta 0:00:03
     --- ------------------------------------ 2.0/23.4 MB 8.5 MB/s eta 0:00:03
     ---- ----------------------------------- 2.4/23.4 MB 9.2 MB/s eta 0:00:03
     ----- ---------------------------------- 3.0/23.4 MB 9.6 MB/s eta 0:00:03
     ----- ---------------------------------- 3.1/23.4 MB 9.6 MB/s eta 0:00:03
     ----- ---------------------------------- 3.1/23.4 MB 9.6 MB/s eta 0:00:03
     ----- ---------------------------------- 3.1/23.4 MB 9.6 MB/s eta 0:00:03
     ----- ---------------------------------- 3.1/23.4 MB 

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      C:\Users\student\AppData\Local\Temp\pip-build-env-u8hwl7m3\overlay\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      C:\Users\student\AppData\Local\Temp\pip-build-env-u8hwl7m3\overlay\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      running dist_info
      creating C:\Users\student\AppData\Local\Temp\pip-modern-metadata-4smdjsei\gensim.egg-info
      writing C:\Users\student\AppData\Local\Temp\pip-modern-metadata-4smdjsei\gensim.egg-info\PKG-INFO
      writing dependency_links to C:\Users\student\AppData\Local\Temp\pip-modern-metadata-4smdjsei\gensim.egg-info\dependency_links.txt
      writing requirements to C:\Users\student\AppData\Local\Temp\pip-

In [26]:
#Reloading the new version of Fasttext model
from gensim.models.fasttext import load_facebook_model

# Load the FastText model
mod = load_facebook_model('tamil_models\cc.ta.300.bin')
model = mod.wv

# Check if a word is present in the model vocabulary
print('sviluppatore' in model.key_to_index)

<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\student\AppData\Local\Temp\ipykernel_8532\2330281311.py:5: SyntaxWarning: invalid escape sequence '\c'
  mod = load_facebook_model('tamil_models\cc.ta.300.bin')


False


In [36]:
for image_file in tqdm(os.listdir(images_folder)):
    image_name = os.path.splitext(image_file)[0]
    
    label_file = os.path.join(labels_folder, f"{image_name}.txt")
    
    # Check if the label file exists
    if not os.path.exists(label_file):
        print(f"Label file not found for {image_file}")
        continue
    
    # Read the label (description)
    with open(label_file, "r", encoding="utf-8") as f:
        description = f.read().strip()
    
    # Generate embedding using FastText
    embedding = generate_embedding_fasttext(description,mod)
    
    # Save embedding as .npy file
    output_file = os.path.join(output_folder, f"{image_name}.npy")
    np.save(output_file, embedding)

100%|██████████| 300/300 [00:00<00:00, 320.23it/s]


#### Glove Model

In [42]:
#I have downloaded and extracted the glove model manually
def load_glove_model(glove_file):
    print("Loading GloVe Model...")
    glove_model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            glove_model[word] = embedding
    print(f"Loaded {len(glove_model)} words.")
    return glove_model
# Load the GloVe model
glove_extracted_path = 'tamil_models\glove.6B\glove.6B.50d.txt'
glove_model = load_glove_model(glove_extracted_path)
print("Glove Model loaded sucessfully")

<>:14: SyntaxWarning: invalid escape sequence '\g'
<>:14: SyntaxWarning: invalid escape sequence '\g'
C:\Users\student\AppData\Local\Temp\ipykernel_8532\895928523.py:14: SyntaxWarning: invalid escape sequence '\g'
  glove_extracted_path = 'tamil_models\glove.6B\glove.6B.50d.txt'


Loading GloVe Model...
Loaded 400000 words.
Glove Model loaded sucessfully


In [43]:
# Function to generate embeddings
def get_embedding(text, glove_model):
    words = text.split()
    embeddings = []
    for word in words:
        if word in glove_model:
            embeddings.append(glove_model[word])
        else:
            # If the word is not in the GloVe model, use a zero vector
            embeddings.append(np.zeros(len(next(iter(glove_model.values())))))
    return np.array(embeddings)

In [44]:
current_folder = r'C:\Users\student\Desktop\Datasets\sample_data'
images_folder = os.path.join(current_folder, "Images")
labels_folder = os.path.join(current_folder, "tamil_labels_300_proofread")
output_folder = os.path.join(current_folder, "embeddings_300_proofread_glove")

In [51]:
from gensim.models import KeyedVectors

# Load pre-trained GloVe vectors (e.g., 50-dimensional vectors)
glove_file = r'tamil_models\glove.6B\glove.6B.50d.txt'  # Replace with the actual path to your GloVe file
glove_model = KeyedVectors.load_word2vec_format(glove_file, binary=False)

# Now you can use the glove_model for word embeddings


ValueError: invalid literal for int() with base 10: 'the'

In [45]:
for image_file in tqdm(os.listdir(images_folder)):
    image_name = os.path.splitext(image_file)[0]
    
    label_file = os.path.join(labels_folder, f"{image_name}.txt")
    
    # Check if the label file exists
    if not os.path.exists(label_file):
        print(f"Label file not found for {image_file}")
        continue
    
    # Read the label (description)
    with open(label_file, "r", encoding="utf-8") as f:
        description = f.read().strip()
    
    # Generate embedding using FastText
    embedding = generate_embedding_fasttext(description,glove_model)
    
    # Save embedding as .npy file
    output_file = os.path.join(output_folder, f"{image_name}.npy")
    np.save(output_file, embedding)

  0%|          | 0/300 [00:00<?, ?it/s]


AttributeError: 'dict' object has no attribute 'wv'